# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
# Imports
import pandas as pd

# Creation of DataFrame
df = pd.read_csv('training.1600000.processed.noemoticon.csv', verbose= True, encoding='ISO-8859-1', header= None)

Tokenization took: 345.82 ms
Type conversion took: 552.56 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 264.51 ms
Type conversion took: 422.03 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 283.55 ms
Type conversion took: 452.39 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 190.05 ms
Type conversion took: 438.65 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 158.93 ms
Type conversion took: 349.90 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 187.45 ms
Type conversion took: 357.89 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 227.04 ms
Type conversion took: 353.01 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 190.97 ms
Type conversion took: 328.07 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 218.78 ms
Type conversion took: 364.77 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 179.45 ms
Type conversion took: 355.35 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 173.77 ms
T

In [2]:
df.columns = ['target','id','date','flag','user','text']
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
# your code here
# Imports
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist

# Seed of random states
seed = 0

# sample of data set
df_sample = df.sample(20000, random_state= seed)

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    # clean URL
    match = re.findall(r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»""'']))', s)
    for m in match:
        url = m[0]
        s = s.replace(url, ' ')
    
    # clean numbers
    for number in re.findall("[0-9]", s):
        s = s.replace(number, ' ')
    
    # clean special characters
    for character in re.findall("[\[@_!#$%^&*()<>?/\|}{~:\-'\".,;:\]]", s):
        s = s.replace(character, ' ')
    
    return s.lower()

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    
    return word_tokenize(s)

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    # Stemming
    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
    
    return lemmatized

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    without_sw = [word for word in l if not word in stopwords.words('english')]
    return without_sw

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df_sample.head()

,target,id,date,flag,user,text
557138,0,2204444171,Wed Jun 17 02:14:00 PDT 2009,NO_QUERY,einmensch,wants to compete! i want hard competition! i w...
349381,0,2017152437,Wed Jun 03 07:56:34 PDT 2009,NO_QUERY,keithmorrison,It seems we are stuck on the ground in Amarill...
182051,0,1967043408,Fri May 29 18:52:13 PDT 2009,NO_QUERY,PunkieDory,where the f are my pinking shears? rarararrrar...
571236,0,2208721054,Wed Jun 17 09:32:48 PDT 2009,NO_QUERY,DYkEY_tYPE,0ff t0 tHE MEEtiN.. i HAtE WhEN PPl V0lUNtEER...
1339637,4,2018731586,Wed Jun 03 10:25:27 PDT 2009,NO_QUERY,BlueSmartiies,@ reply me pls


In [5]:
import time 
start = time.process_time()
df_sample['text_processed'] = df_sample['text'].apply(clean_up)
print('Cleaning performed')  
print(time.process_time() - start)
df_sample['text_processed'] = df_sample['text_processed'].apply(tokenize)
print('Tokenization performed')
print(time.process_time() - start)
df_sample['text_processed'] = df_sample['text_processed'].apply(stem_and_lemmatize)
print('Stemming and lemmatization performed')
print(time.process_time() - start)
df_sample['text_processed'] = df_sample['text_processed'].apply(remove_stopwords)
print('Stopwords removed')
print(time.process_time() - start)

Cleaning performed
0.359375
Tokenization performed
3.921875
Stemming and lemmatization performed
14.015625
Stopwords removed
154.0


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [7]:
# your code here
fdist = FreqDist()

for tweet in df_sample['text_processed']:
    for word in tweet:
        fdist[word] += 1
        
most_common_list = [word[0] for word in fdist.most_common(5000)]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [9]:
# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in most_common_list:
        features[w] = (w in words)

    return features

featuresets = [(find_features(tweet), bool(target)) for (tweet, target) in list(zip(df_sample['text_processed'], df_sample['target']))]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [10]:
# your code here
# set that we'll train our classifier with
training_set = featuresets[:int(len(featuresets)*5/100)] # 19000

# set that we'll test against.
testing_set = featuresets[int(len(featuresets)*5/100):] # 19000

# Training
classifier = nltk.NaiveBayesClassifier.train(training_set)

# most important features
classifier.show_most_informative_features()

Most Informative Features
                    cant = True            False : True   =      8.2 : 1.0
                     whi = True            False : True   =      7.5 : 1.0
                       x = True             True : False  =      7.1 : 1.0
                    dont = True            False : True   =      5.7 : 1.0
                    sick = True            False : True   =      5.6 : 1.0
                    miss = True            False : True   =      5.3 : 1.0
                  happen = True            False : True   =      4.9 : 1.0
                  pretti = True            False : True   =      4.9 : 1.0
                     ugh = True            False : True   =      4.9 : 1.0
                    hate = True            False : True   =      4.5 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [11]:
# your code here
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 65.37894736842105


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [12]:
# your code here
for i in [310]:
    most_common_list = [word[0] for word in fdist.most_common(i)]
    
    featuresets = [(find_features(tweet), bool(target)) for (tweet, target) in list(zip(df_sample['text_processed'], df_sample['target']))]
    
    # set that we'll train our classifier with
    training_set = featuresets[:int(len(featuresets)*5/100)] # 19000

    # set that we'll test against.
    testing_set = featuresets[int(len(featuresets)*5/100):] # 19000

    # Training
    classifier = nltk.NaiveBayesClassifier.train(training_set)

    # most important features
    #display(classifier.show_most_informative_features())
    
    print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
    print('\n')

Classifier accuracy percent: 65.76842105263158




## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [13]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [14]:
# your code here